In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("PySpark") \
    .getOrCreate()

In [128]:
sch = 'idx int,region string,district string,comNum string, totalPpl int, acquiredPaper  int, earlyVote int, paperInPlace int, paperOutPlace int, unusedPaper int, portablePaper int, stationaryPaper int, badPaper int, lostPaper int, missedPaper int,uncheckedPaper int, Baburin float, Grudinin float, Girinovskiy float, Putin float, Sobchak float, Suraikin float, Titov float, Yavlinskiy float' 

In [129]:
df = spark.read.csv("ElectionsData.csv", schema=sch)

In [130]:
df.show(10)

+----+--------------------+----------+---------+--------+-------------+---------+------------+-------------+-----------+-------------+---------------+--------+---------+-----------+--------------+-------+--------+-----------+-----+-------+--------+-----+----------+
| idx|              region|  district|   comNum|totalPpl|acquiredPaper|earlyVote|paperInPlace|paperOutPlace|unusedPaper|portablePaper|stationaryPaper|badPaper|lostPaper|missedPaper|uncheckedPaper|Baburin|Grudinin|Girinovskiy|Putin|Sobchak|Suraikin|Titov|Yavlinskiy|
+----+--------------------+----------+---------+--------+-------------+---------+------------+-------------+-----------+-------------+---------------+--------+---------+-----------+--------------+-------+--------+-----------+-----+-------+--------+-----+----------+
|null|              Регион|     Район|Номер УИК|    null|         null|     null|        null|         null|       null|         null|           null|    null|     null|       null|          null|   nul

In [160]:
df = df.na.drop(subset=["idx"])

In [74]:
print((df.count(), len(df.columns)))

(97695, 23)


In [166]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col

# найти явку (%) по всем регионам, результат отсортировать по убыванию

In [131]:
appDf = df.select(['region','totalPpl','paperInPlace','paperOutPlace']).groupby("region").sum().withColumnRenamed("sum(totalPpl)", "regPpl").withColumnRenamed("sum(paperInPlace)", "regInPaper").withColumnRenamed("sum(paperOutPlace)", "regOutPaper")

In [133]:
appDf.show(2)

+--------------------+------+----------+-----------+
|              region|regPpl|regInPaper|regOutPaper|
+--------------------+------+----------+-----------+
|Республика Саха (...|643786|    418588|      21196|
|Калининградская о...|796445|    472062|      23183|
+--------------------+------+----------+-----------+
only showing top 2 rows



In [138]:
result = appDf.withColumn("appearance", (appDf.regInPaper + appDf.regOutPaper) /appDf.regPpl )

In [216]:
result.select(['region','appearance']).orderBy(result.appearance.desc()).show()

+--------------------+------------------+
|              region|        appearance|
+--------------------+------------------+
|Кабардино-Балкарс...|0.9180157863562131|
|Чеченская Республика|0.9146955829405258|
|Республика Северн...|0.8997431850872647|
|     Республика Тыва|0.8996527738118354|
| Республика Дагестан|0.8747795414462081|
|Карачаево-Черкесс...|0.8740544668203033|
|Территория за пре...|0.8701888804170619|
| Кемеровская область|0.8319785982107322|
|Республика Ингушетия|0.8171131529148902|
|Ямало-Ненецкий ав...|0.8080836409823554|
|    Брянская область| 0.797078003370434|
|   Тюменская область|0.7864934000501499|
|  Краснодарский край|0.7786194847607465|
| Республика Мордовия|0.7785708098460915|
|Республика Татарс...|0.7742157684137118|
|Чувашская Республ...|0.7622296238501433|
|Республика Башкор...|0.7544523455707033|
|Чукотский автоном...|0.7473539846754718|
|  Республика Бурятия|0.7445832262770092|
|Республика Адыгея...|0.7431189118555477|
+--------------------+------------

# выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек)

In [147]:
candid = "Putin"

In [148]:
df.select(['region','district','paperInPlace','paperOutPlace',candid]).where("paperInPlace + paperOutPlace>= 300").orderBy(df[candid].desc()).show(10)

+--------------------+--------------------+------------+-------------+-----+
|              region|            district|paperInPlace|paperOutPlace|Putin|
+--------------------+--------------------+------------+-------------+-----+
|     Республика Тыва|        Улуг-Хемская|         842|           36|100.0|
|     Республика Тыва|        Улуг-Хемская|         770|           45|100.0|
|     Республика Тыва|        Сут-Хольская|         357|           33|100.0|
|     Республика Тыва|        Улуг-Хемская|         339|           19|100.0|
|Республика Татарс...|        Мензелинская|         318|           33|100.0|
|     Республика Тыва|      Бай-Тайгинская|         351|           12|100.0|
| Кемеровская область|Прокопьевск, Цент...|         732|            0|100.0|
| Республика Дагестан|          Шамильская|         410|            8|100.0|
| Республика Дагестан|        Тляратинская|         349|            3|100.0|
| Республика Дагестан|         Цумадинская|         735|           21|100.0|

# найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [161]:
tikdf = df.select(['region','district','totalPpl','paperInPlace','paperOutPlace']).groupby(["region","district"]).sum().withColumnRenamed("sum(totalPpl)", "regPpl").withColumnRenamed("sum(paperInPlace)", "regInPaper").withColumnRenamed("sum(paperOutPlace)", "regOutPaper")

In [162]:
tikdf.show(10)

+--------------------+--------------------+------+----------+-----------+
|              region|            district|regPpl|regInPaper|regOutPaper|
+--------------------+--------------------+------+----------+-----------+
|    Республика Алтай|             Чойская|  6494|      4013|        337|
| Республика Дагестан|          Акушинская| 27253|     24325|        125|
| Республика Калмыкия|         Яшалтинская| 10899|      6627|        483|
|  Республика Карелия|Петрозаводская го...| 99388|     58710|       2219|
|     Республика Крым|Евпаторийская гор...| 87632|     57630|       2532|
|      Алтайский край|       Третьяковская| 10225|      6441|        509|
|   Красноярский край|Бородинская город...| 14169|      8261|        348|
|     Приморский край|        Партизанская| 21138|     12862|        577|
|Астраханская область|         Володарская| 34106|     17547|       1932|
|Волгоградская обл...|         Октябрьская| 15425|      8739|       2203|
+--------------------+----------------

In [163]:
tikdf = tikdf.withColumn("appearance", (tikdf.regInPaper + tikdf.regOutPaper) /tikdf.regPpl )

In [164]:
tikdf.show(3)

+-------------------+-----------+------+----------+-----------+------------------+
|             region|   district|regPpl|regInPaper|regOutPaper|        appearance|
+-------------------+-----------+------+----------+-----------+------------------+
|   Республика Алтай|    Чойская|  6494|      4013|        337|0.6698490914690484|
|Республика Дагестан| Акушинская| 27253|     24325|        125|0.8971489377316259|
|Республика Калмыкия|Яшалтинская| 10899|      6627|        483| 0.652353426919901|
+-------------------+-----------+------+----------+-----------+------------------+
only showing top 3 rows



In [167]:
tikdf.groupby('region').agg(F.max('appearance') - F.min('appearance')).withColumnRenamed("(max(appearance) - min(appearance))", "diff").orderBy(col('diff').desc()).show(10)

+--------------------+-------------------+
|              region|               diff|
+--------------------+-------------------+
|Ямало-Ненецкий ав...| 0.5734044869523003|
|   Красноярский край| 0.4882075422450645|
| Сахалинская область| 0.4842508035844767|
| Республика Дагестан|0.46389878363882775|
|   Самарская область|0.41698130506775954|
| Саратовская область| 0.4003895056872615|
|Республика Татарс...|  0.394714905703235|
|Архангельская обл...|  0.391241494389256|
| Магаданская область| 0.3850900940014431|
|    Хабаровский край| 0.3832921317811984|
+--------------------+-------------------+
only showing top 10 rows



## Добавлю явку в df

In [170]:
df = df.withColumn("appearance", (df.paperInPlace + df.paperOutPlace) /df.totalPpl )

In [171]:
df.show(3)

+---+--------------------+----------+------+--------+-------------+---------+------------+-------------+-----------+-------------+---------------+--------+---------+-----------+--------------+-------+--------+-----------+-----+-------+--------+-----+----------+------------------+
|idx|              region|  district|comNum|totalPpl|acquiredPaper|earlyVote|paperInPlace|paperOutPlace|unusedPaper|portablePaper|stationaryPaper|badPaper|lostPaper|missedPaper|uncheckedPaper|Baburin|Grudinin|Girinovskiy|Putin|Sobchak|Suraikin|Titov|Yavlinskiy|        appearance|
+---+--------------------+----------+------+--------+-------------+---------+------------+-------------+-----------+-------------+---------------+--------+---------+-----------+--------------+-------+--------+-----------+-----+-------+--------+-----+----------+------------------+
|  0|Республика Адыгея...|Адыгейская|УИК №1|    2256|         2181|        0|        2107|           62|         12|           62|           2107|       3|  

# посчитать дисперсию по явке для каждого региона (учитывать УИК)

In [173]:
df.select(['region','appearance']).groupby('region').agg(F.variance('appearance')).withColumnRenamed('var_samp(appearance)','appearance_var').show(20)

+--------------------+--------------------+
|              region|      appearance_var|
+--------------------+--------------------+
|Республика Саха (...|0.017072621360652934|
|Калининградская о...|0.014413259393987382|
|Новосибирская обл...| 0.00996943055812039|
|   город Севастополь|0.004419537898371079|
|Свердловская область|0.007208286333756964|
|  Республика Хакасия|0.013955575325117589|
| Магаданская область|0.029096714771130107|
|Республика Ингушетия|0.006965098215866341|
|Удмуртская Респуб...|0.009327861890636311|
|     Камчатский край|0.029165948367155945|
| Саратовская область|0.023775104233505564|
|Республика Башкор...|0.015090632820637375|
|  Ростовская область|0.017255351339596565|
|   Калужская область|0.013053627939352164|
|Еврейская автоном...|0.011918917193086277|
|Карачаево-Черкесс...|0.015177162489352492|
| Республика Дагестан|0.016731523431915946|
|Ханты-Мансийский ...|0.012467418064671748|
|Республика Татарс...| 0.02002267533396386|
|Ямало-Ненецкий ав...| 0.0361037

# для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [182]:
candid = "Baburin"

In [183]:
df.select(candid).withColumn(f'{candid}Int',df[candid].cast('int')).groupby(f'{candid}Int').count().orderBy(col(f'{candid}Int').desc()).show(100)

+----------+-----+
|BaburinInt|count|
+----------+-----+
|        44|    1|
|        23|    1|
|        21|    1|
|        20|    3|
|        18|    1|
|        17|    1|
|        16|    1|
|        14|    3|
|        13|    1|
|        12|    1|
|        11|    4|
|        10|    9|
|         9|    5|
|         8|   12|
|         7|   20|
|         6|   26|
|         5|   65|
|         4|  136|
|         3|  263|
|         2| 1274|
|         1|15428|
|         0|80439|
+----------+-----+



In [184]:
candid = "Grudinin"

In [185]:
df.select(candid).withColumn(f'{candid}Int',df[candid].cast('int')).groupby(f'{candid}Int').count().orderBy(col(f'{candid}Int').desc()).show(100)

+-----------+-----+
|GrudininInt|count|
+-----------+-----+
|         80|    1|
|         78|    1|
|         75|    1|
|         71|    1|
|         69|    1|
|         68|    2|
|         66|    3|
|         61|    1|
|         60|    1|
|         58|    3|
|         57|    1|
|         56|    5|
|         55|    3|
|         54|    3|
|         53|    5|
|         52|    4|
|         51|    3|
|         50|   14|
|         49|    1|
|         48|   10|
|         47|   10|
|         46|   12|
|         45|   14|
|         44|   18|
|         43|   13|
|         42|   22|
|         41|   20|
|         40|   30|
|         39|   20|
|         38|   38|
|         37|   50|
|         36|   51|
|         35|   67|
|         34|   58|
|         33|   92|
|         32|  109|
|         31|  124|
|         30|  148|
|         29|  178|
|         28|  253|
|         27|  329|
|         26|  405|
|         25|  532|
|         24|  616|
|         23|  858|
|         22|  963|
|         21| 1183|


In [205]:
candid = "Girinovskiy"

In [206]:
df.select(candid).withColumn(f'{candid}Int',df[candid].cast('int')).groupby(f'{candid}Int').count().orderBy(col(f'{candid}Int').desc()).show(100)

+--------------+-----+
|GirinovskiyInt|count|
+--------------+-----+
|            96|    1|
|            78|    1|
|            66|    2|
|            63|    1|
|            60|    1|
|            56|    1|
|            53|    3|
|            50|    1|
|            48|    1|
|            47|    1|
|            46|    2|
|            44|    2|
|            41|    9|
|            40|    8|
|            39|    1|
|            38|    2|
|            37|    6|
|            36|    7|
|            35|   10|
|            34|    2|
|            33|   15|
|            32|    3|
|            31|   11|
|            30|   12|
|            29|   14|
|            28|   16|
|            27|   20|
|            26|   29|
|            25|   54|
|            24|   33|
|            23|   58|
|            22|   70|
|            21|   84|
|            20|  107|
|            19|  112|
|            18|  199|
|            17|  231|
|            16|  343|
|            15|  491|
|            14|  705|
|          

In [207]:
candid = "Putin"

In [208]:
df.select(candid).withColumn(f'{candid}Int',df[candid].cast('int')).groupby(f'{candid}Int').count().orderBy(col(f'{candid}Int').desc()).show(100)

+--------+-----+
|PutinInt|count|
+--------+-----+
|     100|  205|
|      99|  182|
|      98|  508|
|      97|  707|
|      96|  822|
|      95| 1138|
|      94| 1236|
|      93| 1422|
|      92| 1581|
|      91| 1485|
|      90| 1839|
|      89| 1629|
|      88| 1615|
|      87| 1727|
|      86| 2001|
|      85| 2235|
|      84| 2378|
|      83| 2396|
|      82| 2657|
|      81| 2722|
|      80| 3370|
|      79| 3293|
|      78| 3890|
|      77| 4483|
|      76| 5057|
|      75| 5779|
|      74| 5484|
|      73| 5426|
|      72| 5182|
|      71| 4479|
|      70| 3923|
|      69| 3086|
|      68| 2578|
|      67| 2108|
|      66| 1710|
|      65| 1361|
|      64| 1131|
|      63|  999|
|      62|  788|
|      61|  640|
|      60|  504|
|      59|  349|
|      58|  299|
|      57|  260|
|      56|  177|
|      55|  139|
|      54|  119|
|      53|   88|
|      52|   76|
|      51|   46|
|      50|   72|
|      49|   29|
|      48|   20|
|      47|   32|
|      46|   32|
|      45|   1

In [209]:
candid = "Sobchak"

In [210]:
df.select(candid).withColumn(f'{candid}Int',df[candid].cast('int')).groupby(f'{candid}Int').count().orderBy(col(f'{candid}Int').desc()).show(100)

+----------+-----+
|SobchakInt|count|
+----------+-----+
|        36|    1|
|        30|    1|
|        28|    1|
|        27|    1|
|        26|    1|
|        25|    3|
|        23|    4|
|        22|    2|
|        21|    3|
|        19|    4|
|        18|    7|
|        17|    9|
|        16|    6|
|        15|   19|
|        14|   17|
|        13|   15|
|        12|   29|
|        11|   36|
|        10|   61|
|         9|   87|
|         8|  161|
|         7|  333|
|         6|  577|
|         5| 1066|
|         4| 2410|
|         3| 5280|
|         2|11960|
|         1|27526|
|         0|48075|
+----------+-----+



In [211]:
candid = "Suraikin"

In [212]:
df.select(candid).withColumn(f'{candid}Int',df[candid].cast('int')).groupby(f'{candid}Int').count().orderBy(col(f'{candid}Int').desc()).show(100)

+-----------+-----+
|SuraikinInt|count|
+-----------+-----+
|         17|    1|
|         16|    3|
|         15|    1|
|         14|    1|
|         11|   11|
|         10|   35|
|          9|   43|
|          8|   33|
|          7|   42|
|          6|   55|
|          5|  122|
|          4|  177|
|          3|  417|
|          2| 1590|
|          1|15064|
|          0|80100|
+-----------+-----+



In [213]:
candid = "Titov"

In [214]:
df.select(candid).withColumn(f'{candid}Int',df[candid].cast('int')).groupby(f'{candid}Int').count().orderBy(col(f'{candid}Int').desc()).show(100)

+--------+-----+
|TitovInt|count|
+--------+-----+
|      29|    1|
|      23|    1|
|      20|    3|
|      19|    1|
|      17|    1|
|      16|    1|
|      15|    3|
|      14|    4|
|      13|    1|
|      12|    1|
|      11|    2|
|      10|    5|
|       9|    4|
|       8|   13|
|       7|   18|
|       6|   17|
|       5|   41|
|       4|  110|
|       3|  343|
|       2| 2300|
|       1|17737|
|       0|77088|
+--------+-----+

